In [16]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


rng = 69

In [4]:
df = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Transformed.csv")
#df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_All_Games_Transformed.csv")

In [5]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away','win_or_lose_Home','goalsTotal_Away'])

In [6]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [7]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [8]:
corr_matrix['goalsTotal_Home'].sort_values(ascending=False)

goalsTotal_Home              1.000000
goalsAgainstAvg_Home         0.052154
goalsForAvg_Away             0.048619
goalsAgainstAvg_Away         0.040320
reboundsForAvg_Away          0.020040
reboundsForAvg_Home          0.016664
penaltiesForTotal            0.011857
fenwickPercentageAvg_Away    0.003288
penaltiesAgainstTotal        0.001298
pointsFromGameAvg_Away      -0.002941
goalsForAvg_Home            -0.003039
seasonPointsPerGame_Away    -0.006146
hitsDiffAvg_Away            -0.007814
fenwickPercentageAvg_Home   -0.008797
seasonPointsPerGame_Home    -0.022595
pointsFromGameAvg_Home      -0.024717
hitsDiffAvg_Home            -0.029000
Name: goalsTotal_Home, dtype: float64

In [9]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [10]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [11]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('goalsTotal_Home', axis=1)
nhl_labels = train_set['goalsTotal_Home'].copy()

X_train = train_set.drop('goalsTotal_Home', axis=1)
Y_train = train_set['goalsTotal_Home'].copy()

X_valid = train_set.drop('goalsTotal_Home', axis=1)
Y_valid = train_set['goalsTotal_Home'].copy()

X_test = test_set.drop('goalsTotal_Home', axis=1)
Y_test = test_set['goalsTotal_Home'].copy()

df_Final = df_Final.drop('goalsTotal_Home', axis=1)

In [12]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [13]:
# set epochs to use same num in all calls
n_epochs = 50

In [117]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3517 entries, 2920 to 4041
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   seasonPointsPerGame_Home   3517 non-null   float64
 1   goalsForAvg_Home           3517 non-null   float64
 2   goalsAgainstAvg_Home       3517 non-null   float64
 3   hitsDiffAvg_Home           3517 non-null   float64
 4   pointsFromGameAvg_Home     3517 non-null   float64
 5   fenwickPercentageAvg_Home  3517 non-null   float64
 6   reboundsForAvg_Home        3517 non-null   float64
 7   seasonPointsPerGame_Away   3517 non-null   float64
 8   goalsForAvg_Away           3517 non-null   float64
 9   goalsAgainstAvg_Away       3517 non-null   float64
 10  hitsDiffAvg_Away           3517 non-null   float64
 11  pointsFromGameAvg_Away     3517 non-null   float64
 12  fenwickPercentageAvg_Away  3517 non-null   float64
 13  reboundsForAvg_Away        3517 non-null   float64

In [227]:
input_shape=[X_train.shape[1]]

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=input_shape),
    tf.keras.layers.Dense(15, activation='selu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(15, activation='selu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='relu')  # Output layer for regression
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [235]:
def build_model(n_neurons=50, n_hidden=3, dropout_rate=0.4, learning_rate=1e-3, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='selu'))
        model.add(keras.layers.Dropout(dropout_rate))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mae'])
    return model

In [284]:
best_model = build_model(dropout_rate=0.1, learning_rate=0.00077056048175844, n_hidden=2, n_neurons=43)

In [286]:
best_model.summary()

Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_111 (Dense)                    │ (None, 43)                  │             731 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_49 (Dropout)                 │ (None, 43)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_112 (Dense)                    │ (None, 43)                  │           1,892 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_50 (Dropout)                 │ (None, 43)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_113 (Dense)                    │ (None, 1)                   │              44 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,667 (10.42 KB)

 Trainable params: 2,667 (10.42 KB)

 Non-trainable params: 0 (0.00 B)

In [288]:
from scikeras.wrappers import KerasRegressor

# Seems like you need to specify some items again when using the scikeras wrapper
keras_reg = KerasRegressor(model=best_model)


In [290]:
keras_reg.model_.summary()

AttributeError: 'KerasRegressor' object has no attribute 'model_'

In [245]:

from tensorflow.keras.callbacks import EarlyStopping

tf.random.set_seed(42)
# Using a Callback to run all epochs and only save the best one

early_stopping = EarlyStopping(monitor='val_loss', patience=15)
save_file = r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\Best_Over_Under_Model.keras"
checkpoint_cb = keras.callbacks.ModelCheckpoint(save_file, save_best_only = True)


history = keras_reg.fit(X_train_prepared, Y_train, 
              validation_data = (X_valid_prepared, Y_valid),
              callbacks=[early_stopping,checkpoint_cb],epochs=100)

Epoch 1/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 37.1867 - mae: 5.3054 - val_loss: 6.7136 - val_mae: 2.0404
Epoch 2/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13.2642 - mae: 2.8995 - val_loss: 5.7337 - val_mae: 1.9145
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.5200 - mae: 2.4647 - val_loss: 5.7348 - val_mae: 1.9150
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.6240 - mae: 2.3388 - val_loss: 5.6525 - val_mae: 1.9094
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.7467 - mae: 2.2059 - val_loss: 5.7258 - val_mae: 1.9208
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.2988 - mae: 2.1398 - val_loss: 5.5615 - val_mae: 1.8939
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.2508 - mae: 2.1462 - val_loss: 5.5656 - val_mae: 1.8963
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.0011 - mae: 2.1151 - val_loss: 5.5522 - val_mae: 1.8964
Epoch 9/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

In [253]:
from sklearn.model_selection import cross_val_score

# Assuming X and y are your feature matrix and target vector
scores = cross_val_score(keras_reg, X_train_prepared, Y_train, cv=5, scoring='neg_mean_squared_error')

# Convert scores to positive values
mse_scores = -scores
print(f'MSE Scores: {mse_scores}')
print(f'Mean MSE: {np.mean(mse_scores)}')

88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 37.5970 - mae: 5.3195
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 36.7668 - mae: 5.2944
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 38.7242 - mae: 5.4862
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 38.2651 - mae: 5.3677
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 39.4203 - mae: 5.5410
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MSE Scores: [10.2165989  10.36660015 12.05144151  9.23306666 12.37402297]
Mean MSE: 10.848346037265742


In [254]:
X_train_pred = keras_reg.predict(X_train_prepared)
X_test_pred = keras_reg.predict(X_test_prepared)

# Evaluate the model
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(Y_train, X_train_pred)
mae = mean_absolute_error(Y_train, X_train_pred)

print(f'Train MSE: {mse}') # Sensitive to outliers
print(f'Train MAE: {mae}')

mse = mean_squared_error(Y_test, X_test_pred)
mae = mean_absolute_error(Y_test, X_test_pred)

print(f'Test MSE: {mse}') # Sensitive to outliers
print(f'Test MAE: {mae}')

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step
Train MSE: 5.421816803726396
Train MAE: 1.8656833584687573
Test MSE: 5.319217891736668
Test MAE: 1.8843861514871771


In [257]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

AttributeError: 'KerasRegressor' object has no attribute 'history'

In [28]:
df_Predict = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Predict.csv")
#df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_All_Games_Predict.csv")

In [30]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,model.predict(X_real_predict))).T

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 9) + inhomogeneous part.

In [56]:
model.predict(X_real_predict)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


array([[5.8428903],
       [5.8429093],
       [5.8428597],
       [5.8432736],
       [5.843431 ],
       [5.842931 ],
       [5.8440104],
       [5.8433814],
       [5.8429027]], dtype=float32)